### Verimizi okuyalım

In [70]:
import pandas as pd

veriset=pd.read_csv("yorumlar.csv")
print(veriset)

              Firma Adı         İl  \
0         Mr. Çiğ Köfte      Adana   
1                  Köse      Adana   
2     Bolulu Hasan Usta      Adana   
3      Starbucks Coffee      Adana   
4     Şirinoğlu Baklava      Adana   
...                 ...        ...   
3627  Ziyafet Çiğ Köfte  Zonguldak   
3628          Çiğköftem  Zonguldak   
3629        Helvacı Ali  Zonguldak   
3630     Mersin Tantuni  Zonguldak   
3631        Hayal Döner  Zonguldak   

                                               Yorumlar  
0     ['Herşey güzeldi yapanın ellerine sağlık.', 'm...  
1                                                    []  
2     ['Taze ve güzeldi her zamanki kalite şaşırmadı...  
3     ['frozen berry hibiscus yerine mango dragonfru...  
4     ['Beyaz fıstık ilk defa gördüm ', 'Porsiyon aş...  
...                                                 ...  
3627  ['berbat ', '10 dakikaya geldi ve çok lezzetli...  
3628                                                 []  
3629  ['abi tatlı da 

### Her bir firmayı ve bulunduğu ili bir satırda saklarken, firmanın yorumu birden fazla olduğu için Yorumlar sütunu liste biçiminde

### Bunu düzeltmek adına eval ve explode işlemlerini yapıyoruz

In [71]:
veriset["Yorumlar"]=veriset["Yorumlar"].apply(eval)
veri=veriset.explode("Yorumlar").reset_index(drop=True)
print(veri)

           Firma Adı         İl  \
0      Mr. Çiğ Köfte      Adana   
1      Mr. Çiğ Köfte      Adana   
2      Mr. Çiğ Köfte      Adana   
3      Mr. Çiğ Köfte      Adana   
4      Mr. Çiğ Köfte      Adana   
...              ...        ...   
61199    Hayal Döner  Zonguldak   
61200    Hayal Döner  Zonguldak   
61201    Hayal Döner  Zonguldak   
61202    Hayal Döner  Zonguldak   
61203    Hayal Döner  Zonguldak   

                                                Yorumlar  
0                Herşey güzeldi yapanın ellerine sağlık.  
1                                      mükemmel mükemmel  
2          mega çiğ köfte normal çiğ köfte olarak geldi   
3      Çiğköfte çok iyi ama yanındaki yeşillikler bu ...  
4           Bütün garnitürler pörsümüş kötü durumdaydı.   
...                                                  ...  
61199                                   Çok hızlı geldi   
61200         Çok güzeldi bol malzemeli çok teşekkürler   
61201  dürümler güzelde zurnayı neden 2 ye bölüp

### Boş yorum kontrolü ve var ise onları silmek

In [72]:
if veri.isnull().sum().sum()>0:
    veri.dropna(axis=0,inplace=True)

print(veri.isnull().sum())

Firma Adı    0
İl           0
Yorumlar     0
dtype: int64


### Öncelikle temizlemede kullanacağımız Türkçe Stop Words dosyasını okuyoruz

### Bu dosya benim hazırladığımdır. İnternette farklı kaynaklarda benzerini de bulabilirsiniz

In [73]:
with open("stopwturkish.txt","r",encoding="utf-8") as dosya:
    sw=[satir.strip() for satir in dosya.readlines()]

print(sw)

['ama', 'amma', 'anca', 'ancak', 'belki', 'çünkü', 'dahi', 'eğer', 'emme', 'fakat', 'gah', 'gerek', 'hakeza', 'halbuki', 'hatta', 'hele', 'hem', 'hoş', 'ile', 'ile', 'imdi', 'ister', 'kah', 'keşke', 'keza', 'kezalik', 'kim', 'lakin', 'madem', 'mademki', 'mamafih', 'meğer', 'meğerki', 'meğerse', 'netekim', 'neyse', 'nitekim', 'oysa', 'oysaki', 'şayet', 've', 'velev', 'velhasıl', 'velhasılıkelam', 'veya', 'veyahut', 'yahut', 'yalnız', 'yani', 'yok', 'yoksa', 'zira', 'acaba', 'acep', 'açıkça', 'açıkçası', 'adamakıllı', 'adeta', 'bazen', 'bazı', 'bilcümle', 'binaen', 'binaenaleyh', 'bir', 'biraz', 'birazdan', 'birden', 'birden', 'birdenbire', 'birice', 'birlikte', 'bitevi', 'biteviye', 'bittabi', 'bizatihi', 'bizce', 'bizcileyin', 'bizden', 'bizzat', 'boşuna', 'böyle', 'böylece', 'böylecene', 'böylelikle', 'böylemesine', 'böylesine', 'buracıkta', 'burada', 'buradan', 'büsbütün', 'çabuk', 'çabukça', 'çeşitli', 'çoğu', 'çoğun', 'çoğunca', 'çoğunlukla', 'çok', 'çokça', 'çokluk', 'çoklukla', '

### Yorumlarımızı temizlemek için fonksiyon oluşturalım

In [74]:
import re

def temizlik(cumle):
    cumle=str(cumle)  #Doğru temizlemesi açısından tüm yorumları string hale çevir
    temiz=cumle.lower() #Hepsini küçük harf yapar
    temiz=re.sub(r'[^\w\s]'," ",temiz) #Özel karakterleri temizler
    temiz=re.sub(r'\d+'," ",temiz) #Sayıları temizler
    temiz=re.sub(r'\s+'," ",temiz) #Ardışık boşlukları temizler
    temiz=" ".join([kelime for kelime in temiz.split() if kelime not in sw]) #Stop Words olanları temizler
    temiz=temiz.strip() #Baş ve son boşlukları temizler
    return temiz

### Temizlik fonksiyonunu uygulayalım ve Temiz Yorum isimli yeni bir sütuna gönderelim

In [75]:
veri["Temiz Yorum"]=veri["Yorumlar"].apply(temizlik)

print(veri)

           Firma Adı         İl  \
0      Mr. Çiğ Köfte      Adana   
1      Mr. Çiğ Köfte      Adana   
2      Mr. Çiğ Köfte      Adana   
3      Mr. Çiğ Köfte      Adana   
4      Mr. Çiğ Köfte      Adana   
...              ...        ...   
61199    Hayal Döner  Zonguldak   
61200    Hayal Döner  Zonguldak   
61201    Hayal Döner  Zonguldak   
61202    Hayal Döner  Zonguldak   
61203    Hayal Döner  Zonguldak   

                                                Yorumlar  \
0                Herşey güzeldi yapanın ellerine sağlık.   
1                                      mükemmel mükemmel   
2          mega çiğ köfte normal çiğ köfte olarak geldi    
3      Çiğköfte çok iyi ama yanındaki yeşillikler bu ...   
4           Bütün garnitürler pörsümüş kötü durumdaydı.    
...                                                  ...   
61199                                   Çok hızlı geldi    
61200         Çok güzeldi bol malzemeli çok teşekkürler    
61201  dürümler güzelde zurnayı neden 2

### Aynı firma adında aynı ilde aynı yorum varsa onu kaldırır

In [76]:
veri=veri.drop_duplicates(subset=["Firma Adı","İl","Temiz Yorum"],keep=False)
veri.reset_index(drop=True,inplace=True)
print(veri)

           Firma Adı         İl  \
0      Mr. Çiğ Köfte      Adana   
1      Mr. Çiğ Köfte      Adana   
2      Mr. Çiğ Köfte      Adana   
3      Mr. Çiğ Köfte      Adana   
4      Mr. Çiğ Köfte      Adana   
...              ...        ...   
59753    Hayal Döner  Zonguldak   
59754    Hayal Döner  Zonguldak   
59755    Hayal Döner  Zonguldak   
59756    Hayal Döner  Zonguldak   
59757    Hayal Döner  Zonguldak   

                                                Yorumlar  \
0                Herşey güzeldi yapanın ellerine sağlık.   
1                                      mükemmel mükemmel   
2          mega çiğ köfte normal çiğ köfte olarak geldi    
3      Çiğköfte çok iyi ama yanındaki yeşillikler bu ...   
4           Bütün garnitürler pörsümüş kötü durumdaydı.    
...                                                  ...   
59753                                   Çok hızlı geldi    
59754         Çok güzeldi bol malzemeli çok teşekkürler    
59755  dürümler güzelde zurnayı neden 2

### Temiz yorumda tokenize etme

In [77]:
from trtokenizer.tr_tokenizer import WordTokenizer

veri["Tokenize Yorum"]=veri["Temiz Yorum"].apply(WordTokenizer().tokenize)
print(veri)

           Firma Adı         İl  \
0      Mr. Çiğ Köfte      Adana   
1      Mr. Çiğ Köfte      Adana   
2      Mr. Çiğ Köfte      Adana   
3      Mr. Çiğ Köfte      Adana   
4      Mr. Çiğ Köfte      Adana   
...              ...        ...   
59753    Hayal Döner  Zonguldak   
59754    Hayal Döner  Zonguldak   
59755    Hayal Döner  Zonguldak   
59756    Hayal Döner  Zonguldak   
59757    Hayal Döner  Zonguldak   

                                                Yorumlar  \
0                Herşey güzeldi yapanın ellerine sağlık.   
1                                      mükemmel mükemmel   
2          mega çiğ köfte normal çiğ köfte olarak geldi    
3      Çiğköfte çok iyi ama yanındaki yeşillikler bu ...   
4           Bütün garnitürler pörsümüş kötü durumdaydı.    
...                                                  ...   
59753                                   Çok hızlı geldi    
59754         Çok güzeldi bol malzemeli çok teşekkürler    
59755  dürümler güzelde zurnayı neden 2

### Temizlik sonrası veri setimizi kaydedelim

In [78]:
veri.to_csv("temizyorumlar.csv",index=False,encoding="utf-8")